In [ ]:
# Analysing tweets with VADER - a rule- and lexicon based tool for sentiment analysis 

In [1]:
# importing libraries
import re
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#import nltk
from textblob import TextBlob

In [22]:
# import data
#tweets_df = pd.read_pickle('final_data_version_minustop10.pkl')
df_dominant_topic = pd.read_pickle('df_dominant_topic_10_topics.pkl')

# LINE BELOW ADDED AFTER SYNOPSIS HAND IN
# import new updated df with less preprocessed column
tweets_df = pd.read_pickle('oralexamupdated_final_data_version_minustop10.pkl')


In [23]:
# check data 
# print(tweets_df.shape)
#print(df_dominant_topic.shape)
#(8406, 16)
#(9378, 5)

# Remove last rows (they are just there bc there is mismatch with the added textcolumn, the rest is in sync and goes from rows[0-8405])
df_dominant_topic = df_dominant_topic.drop(df_dominant_topic.index[8406:])

tweets_df.reset_index(drop=True, inplace=True)

# concatenate dataframes 
df = pd.concat([tweets_df,df_dominant_topic],axis=1)
# EVT drop last text xcolumn - very confusing! 
# drop "Text" column, not "text" - which is the original tweets

In [10]:
df

,text,location,statuses_count,followers_count,friends_count,listed_count,favourites_count,tidy_text,less_prep_text,tidy_text_tokens,...,no_stopwords_string,lemmas,lemmas_string,stemmed,lemmas_reduced,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,The Future of Vaccines\nhttps://t.co/GI23fycj1...,Earth,11485,774,3240,2,1107,the future never before have billions people b...,The Future of Vaccines\n\nNever before have bi...,"[the, future, never, before, have, billions, p...",...,future never billions people pressured submit,"[never, billion, people, pressure, submit]",never billion people pressure submit,"[futur, never, billion, peopl, pressur, submit]","[never, billion, people, pressure, submit]",0,2.0,0.1255,"hospital, wait, distribute, country, share, de...","[never, billion, people, pressure, submit]"
1,The immunization task force can likely deliver...,"Mississauga, Canada",280,225,675,3,26,the immunization task force can likely deliver...,The immunization task force can likely deliver...,"[the, immunization, task, force, can, likely, ...",...,immunization task force likely deliver shots s...,"[immunization, likely, deliver, shot, small, r...",immunization likely deliver shot small rural h...,"[immun, task, forc, like, deliv, shot, small, ...","[immunization, likely, deliver, shot, small, r...",1,2.0,0.1638,"hospital, wait, distribute, country, share, de...","[immunization, likely, deliver, shot, small, r..."
2,Question if city staff will remain out working...,Toronto,251890,19164,9952,607,30736,question city staff will remain out working wi...,Question if city staff will remain out working...,"[question, city, staff, will, remain, out, wor...",...,question city staff remain working homeless pe...,"[question, city, staff, remain, work, homeless...",question city staff remain work homeless peopl...,"[question, citi, staff, remain, work, homeless...","[question, city, staff, remain, work, homeless...",2,2.0,0.1290,"hospital, wait, distribute, country, share, de...","[question, city, staff, remain, work, homeless..."
3,@iamIqraKhalid @GovCanHealth @moderna_tx Enoug...,"Ontario, Canada",41897,1317,883,5,156959,enough for people big whoppy population canada...,"Enough for 84,000 people! Big Whoppy Do. ...","[enough, for, people, big, whoppy, population,...",...,enough people big whoppy population canada millio,"[enough, people, big]",enough people big,"[enough, peopl, big, whoppi, popul, canada, mi...","[enough, people, big]",3,7.0,0.1154,"people, vaccinate, shot, read, time, develop, ...","[enough, people, big]"
4,.@USForcesKorea will start administering the @...,"Washington, DC",33658,16782,1181,529,558,will start administering the its members this ...,. will start administering the COVID-19 to i...,"[will, start, administering, the, its, members...",...,start administering members week,"[start, administer, member, week]",start administer member,"[start, administ, member, week]","[start, administer, member]",4,0.0,0.1132,"health, worker, care, live, official, administ...","[start, administer, member]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8401,@Lya1224 @SenateDems @PattyMurray Wrong\nThe F...,"Philadelphia, PA",11432,125,207,0,12899,wrong the feds gave billion for advance purcha...,Wrong\nThe Feds gave Pfizer $1.95 BILLION f...,"[wrong, the, feds, gave, billion, for, advance...",...,wrong feds gave billion advance purchase agree...,"[wrong, fed, give, advance, purchase, agreement]",wrong fed give advance purchase agreement,"[wrong, fed, gave, billion, advanc, purchas, a...","[wrong, fed, give, advance, purchase, agreement]",8401,2.0,0.1071,"hospital, wait, distribute, country, share, de...","[vaccinate, placebo, group, people, help, prove]"
8402,A second COVID-19 vaccine has been authorized ...,"Cullman, AL",5806,25,145,0,3,second has been authorized for emergency use c...,A second COVID-19 has been authorized for eme...,"[second, has, been, authorized, for, emerge

In [24]:
df.shape

(8406, 21)

In [28]:
#VADER Sentiment Analysis
# Get/call SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#Add VADER metrics to dataframe
#df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['lemmas_string']]
#df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in df['lemmas_string']]
#df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in df['lemmas_string']]
#df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in df['lemmas_string']]
#df.head(3)


# CODE BELOW ADDED AFTER SYNOPSIS HAND IN 
df['new_compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['less_prep_text']]
df['new_neg'] = [analyzer.polarity_scores(v)['neg'] for v in df['less_prep_text']]
df['new_neu'] = [analyzer.polarity_scores(v)['neu'] for v in df['less_prep_text']]
df['new_pos'] = [analyzer.polarity_scores(v)['pos'] for v in df['less_prep_text']]
#df.head(3)

In [29]:
# neg, neu, pos are multidimensional measures of text sentiment, they each represent what ratio of the text that belong to each category 
# compound score is the recommended metric if you want to have a unidimensional measure of text sentiment. It is a "normalized, wegihted composite score" which is derived the lexicon words' valence scores that is adjusted and normalized between -1 (mega neg) and 1 (mega pos). 
# see more info here: https://github.com/cjhutto/vaderSentiment

# according to the github repo reffered above, it is 
# "useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:
# positive sentiment: compound score >= 0.05
# neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
# negative sentiment: compound score <= -0.05" 

# refers to turning compound scores into classes pos, neu or neg
# POS = compound score >= 0.05
# NEU = -0.05 < compoundscore < 0.05
# NEG = compound score <= -0.05

# get sentiment scores:
#sentiment = []
#for row in df['compound']:
#    if row >= 0.05:                     sentiment.append('POS')
#    elif row > -0.05 and row < 0.05:    sentiment.append('NEU')
#    elif row <= -0.05:                  sentiment.append('NEG')
#    else:                               sentiment.append('Not_Rated')

# CODE BELOW ADDED AFTER SYNOPSIS HAND IN
# get sentiment scores:
new_sentiment = []
for row in df['new_compound']:
    if row >= 0.05:                     new_sentiment.append('POS')
    elif row > -0.05 and row < 0.05:    new_sentiment.append('NEU')
    elif row <= -0.05:                  new_sentiment.append('NEG')
    else:                               new_sentiment.append('Not_Rated')


In [30]:
#df['sentiment'] = sentiment
df['new_sentiment'] = new_sentiment
print(df)

                                                   text             location  \
0     The Future of Vaccines\nhttps://t.co/GI23fycj1...                Earth   
1     The immunization task force can likely deliver...  Mississauga, Canada   
2     Question if city staff will remain out working...              Toronto   
3     @iamIqraKhalid @GovCanHealth @moderna_tx Enoug...      Ontario, Canada   
4     .@USForcesKorea will start administering the @...       Washington, DC   
...                                                 ...                  ...   
8401  @Lya1224 @SenateDems @PattyMurray Wrong\nThe F...     Philadelphia, PA   
8402  A second COVID-19 vaccine has been authorized ...          Cullman, AL   
8403  @no_silenced They already have...\nExhibit A: ...                   NC   
8404  F.D.A. Authorizes Moderna #Coronavirus Vaccine...             Montréal   
8405  @atrupar He's throttling back Pfizer's vaccine...         Homewood, AL   

      statuses_count  followers_count  

In [31]:
# save df - main df
# save to csv file:
#df.to_csv('final_main_df_topics_sentiment_added.csv', header=True, mode='a')
# save to pickle 
#df.to_pickle('final_main_df_topics_sentiment_added.pkl')

# CODE BELOW ADDED AFTER SYNOPSIS HAND IN
# save to csv file:
df.to_csv('oralexamupdated_final_main_df_topics_sentiment_added.csv', header=True, mode='a')
# save to pickle 
df.to_pickle('oralexamupdated_final_main_df_topics_sentiment_added.pkl')



In [53]:
#test = df1.stack()
test.reset_index()

,Dominant_Topic,sentiment,0
0,0.0,NEG,152
1,0.0,NEU,615
2,0.0,POS,396
3,1.0,NEG,186
4,1.0,NEU,447
5,1.0,POS,408
6,2.0,NEG,119
7,2.0,NEU,512
8,2.0,POS,312
9,3.0,NEG,246


In [48]:
#df.groupby('Dominant_Topic')['sentiment'].value_counts()
dftest = df.groupby('Dominant_Topic')['sentiment'].value_counts().fillna(0)

"""
Dominant_Topic  sentiment
0.0             NEU          568
                POS          469
                NEG          126
1.0             POS          447
                NEU          401
                NEG          193
2.0             NEU          433
                POS          381
                NEG          129
3.0             NEU          464
                POS          288
                NEG          244
4.0             NEU          382
                POS          295
                NEG           94
5.0             NEU          349
                POS          308
                NEG          115
6.0             POS          279
                NEU          241
                NEG          222
7.0             POS          288
                NEU          259
                NEG          129
8.0             POS          309
                NEU          242
                NEG          120
9.0             NEU          294
                POS          236
                NEG          101
"""

'\nDominant_Topic  sentiment\n0.0             NEU          568\n                POS          469\n                NEG          126\n1.0             POS          447\n                NEU          401\n                NEG          193\n2.0             NEU          433\n                POS          381\n                NEG          129\n3.0             NEU          464\n                POS          288\n                NEG          244\n4.0             NEU          382\n                POS          295\n                NEG           94\n5.0             NEU          349\n                POS          308\n                NEG          115\n6.0             POS          279\n                NEU          241\n                NEG          222\n7.0             POS          288\n                NEU          259\n                NEG          129\n8.0             POS          309\n                NEU          242\n                NEG          120\n9.0             NEU          294\n                PO

In [39]:
df1 = df.groupby('Dominant_Topic')['sentiment'].value_counts().unstack().fillna(0)

# LINE BELOW ADDED AFTER SYNOPSIS HAND IN
df1_new = df.groupby('Dominant_Topic')['new_sentiment'].value_counts().unstack().fillna(0)

In [40]:
print(df1_new)

new_sentiment   NEG  NEU  POS
Dominant_Topic               
0.0             130  580  453
1.0             206  389  446
2.0             143  443  357
3.0             253  452  291
4.0             108  386  277
5.0             123  350  299
6.0             236  246  260
7.0             143  242  291
8.0             124  238  309
9.0             111  306  214


In [ ]:
df1:
sentiment       NEG  NEU  POS
Dominant_Topic               
0.0             152  615  396
1.0             186  447  408
2.0             119  512  312
3.0             246  485  265
4.0             105  447  219
5.0             121  378  273
6.0             198  323  221
7.0             137  288  251
8.0             122  264  285
9.0             107  347  177

df1_new:
new_sentiment   NEG  NEU  POS
Dominant_Topic               
0.0             130  580  453
1.0             206  389  446
2.0             143  443  357
3.0             253  452  291
4.0             108  386  277
5.0             123  350  299
6.0             236  246  260
7.0             143  242  291
8.0             124  238  309
9.0             111  306  214

In [41]:
df2 = df.groupby('sentiment')['Dominant_Topic'].value_counts().unstack().fillna(0)

#df2
#Dominant_Topic  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
#sentiment                                                       
#NEG             152  186  119  246  105  121  198  137  122  107
#NEU             615  447  512  485  447  378  323  288  264  347
#POS             396  408  312  265  219  273  221  251  285  177

# LINE BELOW ADDED AFTER SYNOPSIS HAND IN
df2_new = df.groupby('new_sentiment')['Dominant_Topic'].value_counts().unstack().fillna(0)

#df2_new
#Dominant_Topic  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
#new_sentiment                                                   
#NEG             130  206  143  253  108  123  236  143  124  111
#NEU             580  389  443  452  386  350  246  242  238  306
#POS             453  446  357  291  277  299  260  291  309  214


In [42]:
print(df2_new)

Dominant_Topic  0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
new_sentiment                                                   
NEG             130  206  143  253  108  123  236  143  124  111
NEU             580  389  443  452  386  350  246  242  238  306
POS             453  446  357  291  277  299  260  291  309  214


In [58]:
# BLOCK ADDED AFTER SYNOPSIS HAND IN
# create visualisation of how the distribution between neg, neu, pos has changed with the less preprocessed text version 
# but first, prepare data to plot 

# data to use 
# df1 
# df1_new

def prep_df(df_name, df_callname):
    df_name = df_name.stack().reset_index()
    df_name.columns = ['Dominant_Topic', 'Sentiment_Cat', 'Values']
    df_name['Text_Version'] = df_callname
    return df_name

plot_df1 = prep_df(df1, 'Original')
plot_df1_new = prep_df(df1_new, 'Updated')

df_test_plot = pd.concat([plot_df1, plot_df1_new])

In [61]:
# BLOCK ADDED AFTER SYNOPSIS HAND IN
#import pandas as pd
#import numpy as np
#import altair as alt

alt.Chart(df_test_plot).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('Text_Version:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(Values):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('Dominant_Topic:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Sentiment_Cat:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#e25822', '#FCCE45', '#88B04B'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

alt.Chart(...)

In [68]:
#save df2
df2.to_csv('df_groupedby_sentiment_dominanttopic.csv', header=True, mode='a')
df2.to_csv('minus_amode_df_groupedby_sentiment_dominanttopic.csv', header=True)


In [68]:
# THIS BLOCK IS ADDED AFTER SYNOPSIS HAND IN

# Creating a subset of the lemmas_string/text column and sentiment column to inspect/evaluate performance of the sentiment model
#sentiment_eval_subset = df[['lemmas_string', 'sentiment', "less_prep_text", "new_sentiment"]]
sentiment_eval_subset.to_csv('sentiment_eval_subset.csv', header=True, mode='a')



In [14]:
#### Textblob analysis example as inspired by Medium article ####
# (https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d)
# I am not using this part of the code

# TextBlob Sentiment Analysis
# Load the text/tweets column as a TextBlob, and add two new columns for polarity and subjectivity
#load the text into textblob
text_blob = [TextBlob(text) for text in tweets_df['text']]
#add the sentiment metrics to the dataframe
tweets_df['tb_Pol'] = [b.sentiment.polarity for b in text_blob]
tweets_df['tb_Subj'] = [b.sentiment.subjectivity for b in text_blob]
#show dataframe
tweets_df.head(3)

,text,user,location,statuses_count,user_description,followers_count,friends_count,listed_count,favourites_count,tidy_text,tidy_text_tokens,tokens_no_stopwords,no_stopwords_string,lemmas,lemmas_string,stemmed,stemmed_string,tb_Pol,tb_Subj
0,@Gunntwitt I saw my military neurologist yeste...,kleegrubaugh,Texas Hill Country,172187,First - Be kind. Reformed Christian. Conservat...,3883,4978,90,162046,saw military neurologist yesterday sammc san a...,"[saw, military, neurologist, yesterday, sammc,...","[military, neurologist, yesterday, sammc, san,...",military neurologist yesterday sammc san anton...,[center],center,"[militari, neurologist, yesterday, sammc, san,...",militari neurologist yesterday sammc san anton...,-0.075,0.075
1,$MRK Announces Sale of its Direct Equity Inves...,BioStocks,Toronto,45006,Investor/trader with over a decade of experien...,40945,76,912,301,mrk announces sale its direct equity investmen...,"[mrk, announces, sale, its, direct, equity, in...","[mrk, announces, sale, direct, equity, investm...",mrk announces sale direct equity investment mrna,"[announce, sale, direct, equity, investment]",announce sale direct equity investment,"[mrk, announc, sale, direct, equiti, invest, m...",mrk announc sale direct equiti invest mrna,0.100,0.400
2,‘Absolutely remarkable’: No one who got Modern...,MJosling53,Canada,68026,Former IDF bomb disposal unit member including...,1272,1993,2,62456,absolutely remarkable one who got trial develo...,"[absolutely, remarkable, one, who, got, trial,...","[absolutely, remarkable, trial, developed, sev...",absolutely remarkable trial developed severe,"[absolutely_remarkable, trial, develop, severe]",absolutely_remarkable trial develop severe,"[absolut, remark, trial, develop, sever]",absolut remark trial develop sever,0.425,0.525


In [5]:
# testing textblob
# # Preparing an input sentence
sentence = "today panel will decide who gets the first umd docs thrilled with reported effec."

# Creating a textblob object assignign sentiment
analysis = TextBlob(sentence).sentiment
print(analysis)
#output is a namedtuple of the form Sentiment(polarity, subjectivity)
# you can also do: 
#analysisPol = TextBlob(sentence).polarity
#analysisSub = TextBlob(sentence).subjectivity
# if you want to display polarity and subjectivity seperately...
#print(analysisPol)
#print(analysisSub)

# you can use textblob with different algorithms
#PatternAnalyzer - default classifier built on the pattern library
#NaiveBayesAnalyzer - NLTK model trained on a corpus of movie reviews

Sentiment(polarity=0.425, subjectivity=0.5166666666666666)
